In [14]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")
sys.path.append("../")
sys.path.append("/media/data/GitHub/simple_hwr")
import os
import numpy as np
from hwr_utils import *
from hwr_utils.stroke_plotting import *
from hwr_utils.stroke_recovery import get_number_of_stroke_pts_from_gt
from hwr_utils.stroke_recovery import *
import json
from matplotlib import pyplot as plt

In [2]:
import numpy as np

x = np.array([1,2])
y = x
x = x[::-1]
print(y)

[1 2]


In [5]:
gt = np.array(range(36)).reshape(9,4)
gt[:,2] = [1,0,0,1,0,1,1,0,0]
print(gt)

[[ 0  1  1  3]
 [ 4  5  0  7]
 [ 8  9  0 11]
 [12 13  1 15]
 [16 17  0 19]
 [20 21  1 23]
 [24 25  1 27]
 [28 29  0 31]
 [32 33  0 35]]


In [6]:
def swap_strokes_left(gt, stroke_numbers=False, sos_index=2, distance_threshold=.3):
    """ Swap stroke if the most left part is further right than the previous stroke

    Args:
        gt:
        stroke_numbers:
        sos_index:

    Returns:

    """
    # gt = np.array(range(36)).reshape(9, 4)
    # gt[:, 2] = [1, 0, 0, 1, 0, 1, 0, 0, 0]
    # gt[-1, :] = [3.4, 5, 1, 1]
    # print(gt)

    sos_args = get_sos_args(gt[:, sos_index], stroke_numbers=stroke_numbers)
    strokes = np.split(gt, sos_args)[1:]

    min_x_position = [np.min(s[:, 0]) for s in strokes if s.size]
    min_x_position += np.cumsum([distance_threshold] * len(min_x_position))  # this acts like a threshold, but increases for larger stroke swaps

    x = list(zip(strokes, min_x_position))
    x.sort(key=lambda m: m[1])

    output = np.concatenate([i[0] for i in x])

    if stroke_numbers: # Stroke numbers are now out of order; find where the strokes change, then re-add
        sos = stroke_recovery.relativefy(output[:,2])!=0
        output[:,2] = np.cumsum(sos) # Regenerate stroke numbers

    return output

swap_strokes_left(gt)

array([[ 0,  1,  1,  3],
       [ 4,  5,  0,  7],
       [ 8,  9,  0, 11],
       [12, 13,  1, 15],
       [16, 17,  0, 19],
       [20, 21,  1, 23],
       [24, 25,  1, 27],
       [28, 29,  0, 31],
       [32, 33,  0, 35]])

In [8]:
list(gt_to_pil(gt))

[[(0, 1), (4, 5), (8, 9)],
 [(12, 13), (16, 17)],
 [(20, 21)],
 [(24, 25), (28, 29), (32, 33)]]

In [85]:
def swap_items(l, pos1, pos2): 
    l[pos1], l[pos2] = l[pos2], l[pos1] 
    return l

def swap_strokes(gt, start, end, pivot):
    """
        pivot: index of stroke break
    """
    t = np.copy(gt[start:end])
    gt[start:start+end-pivot] = t[pivot-start:end-start] 
    gt[start+end-pivot:end] = t[:pivot-start]
    return gt

def swap_to_minimize_l1(pred, gt):
    gt_stroke_lens = get_number_of_stroke_pts_from_gt(gt)
    
    pos = 0
    for i in range(len(gt_stroke_lens)-1):
        print(i)
        end = pos+gt_stroke_lens[i]+gt_stroke_lens[i+1]
        normal_slice = slice(pos,end)
        normal_l1 = abs(gt[normal_slice,:2] - pred[normal_slice,:2])
        
        alt_gt = np.concatenate([gt[pos+gt_stroke_lens[i]:end,:2], gt[pos:pos+gt_stroke_lens[i], :2]])
        alternative_l1 = abs(alt_gt - pred[normal_slice, :2])
#         if i == 0:
#             print(alt_gt)
#             print(alternative_l1)
#             print(normal_l1)
        if np.sum(alternative_l1) < np.sum(normal_l1):
            print(f"Swap {i} {i+1}")
            gt = swap_strokes(gt, pos, end, pivot=pos+gt_stroke_lens[i])
            
            swap_items(gt_stroke_lens, i, i+1)
            pass
        pos += gt_stroke_lens[i]
#     print(gt)
    return gt
    
    
gt = np.array(range(36)).reshape(9,4)
gt[:,2] = [1,0,0,1,0,1,1,0,0]
print(gt)

preds = [[24, 25, 1, 27], [28, 29, 0, 31], [32, 33, 1, 35],[20, 21, 1, 23], [12, 13, 1, 15], [16, 17, 0, 19], [0, 1, 1, 3], [4, 5, 0, 7], [8, 9, 0, 11]]
preds = np.asarray(preds)
print(preds)

[[ 0  1  1  3]
 [ 4  5  0  7]
 [ 8  9  0 11]
 [12 13  1 15]
 [16 17  0 19]
 [20 21  1 23]
 [24 25  1 27]
 [28 29  0 31]
 [32 33  0 35]]
[[24 25  1 27]
 [28 29  0 31]
 [32 33  1 35]
 [20 21  1 23]
 [12 13  1 15]
 [16 17  0 19]
 [ 0  1  1  3]
 [ 4  5  0  7]
 [ 8  9  0 11]]


In [86]:
gt=swap_to_minimize_l1(preds, gt)

0
[[12 13]
 [16 17]
 [ 0  1]
 [ 4  5]
 [ 8  9]]
[[12 12]
 [12 12]
 [32 32]
 [16 16]
 [ 4  4]]
[[24 24]
 [24 24]
 [24 24]
 [ 8  8]
 [ 4  4]]
Swap 0 1
1
Swap 1 2
2
Swap 2 3
[[12 13  1 15]
 [16 17  0 19]
 [20 21  1 23]
 [24 25  1 27]
 [28 29  0 31]
 [32 33  0 35]
 [ 0  1  1  3]
 [ 4  5  0  7]
 [ 8  9  0 11]]


In [87]:
swap_to_minimize_l1(preds, gt)

0
[[20 21]
 [12 13]
 [16 17]]
[[ 4  4]
 [16 16]
 [16 16]]
[[12 12]
 [12 12]
 [12 12]]
1
Swap 1 2
2
[[12 13  1 15]
 [16 17  0 19]
 [24 25  1 27]
 [28 29  0 31]
 [32 33  0 35]
 [20 21  1 23]
 [ 0  1  1  3]
 [ 4  5  0  7]
 [ 8  9  0 11]]


array([[12, 13,  1, 15],
       [16, 17,  0, 19],
       [24, 25,  1, 27],
       [28, 29,  0, 31],
       [32, 33,  0, 35],
       [20, 21,  1, 23],
       [ 0,  1,  1,  3],
       [ 4,  5,  0,  7],
       [ 8,  9,  0, 11]])

In [88]:
print(preds)

[[24 25  1 27]
 [28 29  0 31]
 [32 33  1 35]
 [20 21  1 23]
 [12 13  1 15]
 [16 17  0 19]
 [ 0  1  1  3]
 [ 4  5  0  7]
 [ 8  9  0 11]]


In [89]:
def sort_strokes(gt, original_sort, new_sort):
    gt_list = gt_to_list_of_strokes(gt, stroke_number=False)
    zip_list = zip(gt_list, original_sort)
    

In [94]:
import torch
from torch import tensor
x = tensor(gt.astype('float')).requires_grad_(True)-gt
x.requires_grad

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.